# A1111 Model Installer — SD 1.5 (UI)

Install **Stable Diffusion 1.5** models and common extension assets into the WebUI data directory.

**Folders (under `WEBUI_ROOT`, default `/workspace/a1111`):**
- **Checkpoints** → `models/Stable-diffusion/`
- **VAE** → `models/VAE/`
- **ControlNet (SD1.5)** → `models/ControlNet/`
- **ADetailer YOLO** → `models/adetailer/`
- **ESRGAN** → `models/ESRGAN/`
- **IP-Adapter** → `models/ControlNet/` (used via ControlNet)
- **AnimateDiff** → `models/AnimateDiff/`
- **ReActor (InsightFace)** → `models/insightface/`
- **Embeddings** → `embeddings/`
- **LoRA** → `models/Lora/`

Re-run any time — existing files are skipped.

> Optional: set `HF_TOKEN` (for gated HF models) and/or `CIVITAI_TOKEN` (for private/authorized Civitai links). Notebook still works without them.

In [ ]:
from pathlib import Path
import os

ROOT = Path(os.environ.get("WEBUI_ROOT", "/workspace/a1111"))
CKPT_DIR    = ROOT / "models/Stable-diffusion"
VAE_DIR     = ROOT / "models/VAE"
CN_DIR      = ROOT / "models/ControlNet"
AD_DIR      = ROOT / "models/adetailer"
ESRGAN_DIR  = ROOT / "models/ESRGAN"
IPAD_DIR    = CN_DIR
ANIM_DIR    = ROOT / "models/AnimateDiff"
REACT_DIR   = ROOT / "models/insightface"
EMB_DIR     = ROOT / "embeddings"
LORA_DIR    = ROOT / "models/Lora"

for d in (CKPT_DIR, VAE_DIR, CN_DIR, AD_DIR, ESRGAN_DIR, IPAD_DIR, ANIM_DIR, REACT_DIR, EMB_DIR, LORA_DIR):
    d.mkdir(parents=True, exist_ok=True)

print("Data root:", ROOT)
print("↳ Checkpoints:", CKPT_DIR)
print("↳ VAE:", VAE_DIR)
print("↳ ControlNet + IP-Adapter:", CN_DIR)
print("↳ ADetailer:", AD_DIR)
print("↳ ESRGAN:", ESRGAN_DIR)
print("↳ AnimateDiff:", ANIM_DIR)
print("↳ ReActor (InsightFace):", REACT_DIR)
print("↳ Embeddings:", EMB_DIR)
print("↳ LoRA:", LORA_DIR)

## Helpers (progress + smart filenames)
Checkbox items can be **Hugging Face** (repo+filename) or a **direct URL** (e.g., Civitai). The downloader auto-detects and uses the right method.

In [ ]:
import os, re, shutil, requests
from urllib.parse import urlparse
from tqdm import tqdm
from huggingface_hub import hf_hub_download
from pathlib import Path

def human(n: int) -> str:
    for u in ("B","KB","MB","GB"):
        if n < 1024: return f"{n:.1f} {u}"
        n /= 1024
    return f"{n:.1f} TB"

def _save_stream_to(dest: Path, r: requests.Response) -> Path:
    dest.parent.mkdir(parents=True, exist_ok=True)
    if dest.exists():
        print("✔ Exists:", dest); return dest
    total = int(r.headers.get("Content-Length", 0))
    tmp = dest.with_suffix(dest.suffix + ".part")
    with open(tmp, "wb") as f, tqdm(total=total, unit="B", unit_scale=True, desc=dest.name, ncols=80) as bar:
        for chunk in r.iter_content(chunk_size=1024*1024):
            if chunk:
                f.write(chunk); bar.update(len(chunk))
    tmp.replace(dest)
    print(f"✔ Saved: {dest} ({human(dest.stat().st_size)})")
    return dest

def _filename_from_headers_or_url(r: requests.Response, url: str, default_ext: str = ".safetensors") -> str:
    cd = r.headers.get("Content-Disposition", "")
    m = re.search(r'filename="?([^";]+)"?', cd)
    if m:
        return m.group(1)
    name = url.split("/")[-1].split("?", 1)[0]
    if "." not in name:
        name += default_ext
    return name

def download_direct(url: str, dest_dir: Path, dest_name: str|None=None) -> Path:
    headers = {"User-Agent": "sd-webui-hub-notebook/1.0"}
    netloc = urlparse(url).netloc.lower()
    civitai_token = os.environ.get("CIVITAI_TOKEN")
    if "civitai.com" in netloc and civitai_token:
        headers["Authorization"] = f"Bearer {civitai_token}"
    r = requests.get(url, stream=True, allow_redirects=True, headers=headers)
    r.raise_for_status()
    filename = dest_name or _filename_from_headers_or_url(r, url)
    return _save_stream_to(dest_dir/filename, r)

def download_hf(repo_id: str, filename: str, dest_dir: Path, dest_name: str|None=None) -> Path:
    dest = dest_dir / (dest_name or filename)
    if dest.exists():
        print("✔ Exists:", dest); return dest
    print(f"→ HF: {repo_id}/{filename}")
    path = hf_hub_download(
        repo_id=repo_id,
        filename=filename,
        token=os.environ.get("HF_TOKEN"),
        resume_download=True
    )
    shutil.copy2(path, dest)
    print(f"✔ HF saved: {dest} ({human(dest.stat().st_size)})")
    return dest

def fetch(item: dict, dest_dir: Path) -> None:
    if "hf" in item:
        repo, fname = item["hf"]["repo"], item["hf"]["filename"]
        download_hf(repo, fname, dest_dir, item.get("dest_name"))
    elif "url" in item:
        download_direct(item["url"], dest_dir, item.get("dest_name"))
    else:
        raise ValueError("Item must contain 'hf' or 'url'.")

def download_extras(lines: str, dest_dir: Path) -> None:
    for url in [u.strip() for u in lines.splitlines() if u.strip()]:
        try:
            download_direct(url, dest_dir)
        except Exception as e:
            print("✖", url, "→", e)

## LoRA (example additions)

In [ ]:
import ipywidgets as W
from IPython.display import display, Markdown

lora_items = [
  {"name": "RealisticVision+ Enhancer (LoRA)", "url": "https://civitai.com/api/download/models/712234?type=Model&format=SafeTensor", "dest_name": "RealisticVisionEnhancer.safetensors"},
  {"name": "Detail Tweaker (LoRA)", "url": "https://civitai.com/api/download/models/715574?type=Model&format=SafeTensor", "dest_name": "DetailTweaker.safetensors"}
]
lora_pre = {"RealisticVision+ Enhancer (LoRA)", "Detail Tweaker (LoRA)"}
lora_checks = [W.Checkbox(description=i["name"], value=(i["name"] in lora_pre)) for i in lora_items]
lora_extra = W.Textarea(placeholder="Extra LoRA URLs (.safetensors) — one per line", layout=W.Layout(width="100%", height="80px"))

btn = W.Button(description="Download Selected", button_style="success", icon="download")
out = W.Output()

def on_click(_):
    out.clear_output()
    with out:
        for cb, item in zip(lora_checks, lora_items):
            if cb.value:
                fetch(item, LORA_DIR)
        download_extras(lora_extra.value, LORA_DIR)
        print("✔ LoRA downloads complete.")

btn.on_click(on_click)

display(Markdown("### LoRA Downloads"))
for w in lora_checks: display(w)
display(Markdown("**Extra URLs**")); display(lora_extra)
display(btn, out)